In [1]:
import sqlite3


# connnect to the database

conn = sqlite3.connect("mydb.db")
cursor = conn.cursor()

cursor.execute(
    """CREATE TABLE IF NOT EXISTS students (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER,
        address TEXT
    )"""
)

In [2]:
cursor.execute("")

In [3]:
cursor.execute("INSERT INTO students (name , age , address ) VALUES(? , ? , ?)", ("Angel", 11 , "Ithari ko pul ma "))
conn.commit()

In [5]:
cursor.execute("INSERT INTO students (name , age , address) VALUES(?, ?, ?)", ("Manav", 41, "Dharan"))
conn.commit()

In [8]:
cursor.execute("INSERT INTO students (name, age, address) VALUES(?, ?, ?)", ("Mausam", 20, "Kerabari"))
conn.commit()

In [9]:
cursor.execute("SELECT * FROM students")
rows = cursor.fetchall()   # yesle chai result dinxa

print(rows)  # it gives list of tuples

[(1, 'Angel', 11, 'Ithari ko pul ma '), (2, 'Manav', 41, 'Dharan'), (3, 'Mausam', 20, 'Kerabari'), (4, 'Mausam', 20, 'Kerabari')]


In [11]:
cursor.execute("DELETE FROM students WHERE id = ?", (4,))
conn.commit()

In [12]:
cursor.execute("SELECT * FROM students")
rows = cursor.fetchall()   # yesle chai result dinxa

print(rows)  # it gives list of tuples

[(1, 'Angel', 11, 'Ithari ko pul ma '), (2, 'Manav', 41, 'Dharan'), (3, 'Mausam', 20, 'Kerabari')]


In [13]:
sortedRows = sorted(rows , key= lambda x:x[1])
print(sortedRows)

[(1, 'Angel', 11, 'Ithari ko pul ma '), (2, 'Manav', 41, 'Dharan'), (3, 'Mausam', 20, 'Kerabari')]


In [16]:
# to select ithari only from students



cursor.execute('SELECT * FROM students WHERE address="Dharan"')
print(cursor.fetchall())

[(2, 'Manav', 41, 'Dharan')]


In [17]:
actualRows = cursor.fetchall()
print(actualRows)

[]


In [21]:
cursor.execute('SELECT * FROM students WHERE address LIKE "%n" ')
rows = cursor.fetchall()
print(rows)

[(2, 'Manav', 41, 'Dharan')]


In [1]:
# ORM = Object Relational Mapping

# SQLAlchemy -> interact with databases with two approach. 
# 1 . python like syntax use garera nei hamile db operation use garna sakinxa.
# 2. also using the sql queries 

from sqlalchemy import create_engine

engine = create_engine('sqlite:///anotherdb.db', echo=True)

In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column , Integer , String , DateTime


Base = declarative_base()  # yesbata baneko obj bata student le inherit garxa.

class Student(Base):
    __tablename__ = 'students'
    id = Column(Integer , primary_key=True)
    name = Column(String)
    address = Column(String)
    age = Column(Integer)
    

# Now how to create a table. 
Base.metadata.create_all(engine)

2026-01-18 08:52:13,566 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-18 08:52:13,567 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2026-01-18 08:52:13,568 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-01-18 08:52:13,570 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("students")
2026-01-18 08:52:13,571 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-01-18 08:52:13,573 INFO sqlalchemy.engine.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	address VARCHAR, 
	age INTEGER, 
	PRIMARY KEY (id)
)


2026-01-18 08:52:13,574 INFO sqlalchemy.engine.Engine [no key 0.00106s] ()
2026-01-18 08:52:13,586 INFO sqlalchemy.engine.Engine COMMIT


C:\Users\DELL\AppData\Local\Temp\ipykernel_14984\549188834.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()  # yesbata baneko obj bata student le inherit garxa.


In [3]:
# now we have to create a session 

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

session = Session()

In [4]:
student1 = Student(name="Ram" , age=20 , address = "Dharan")
student2 = Student(name="Angel" , age=26 , address = "Jhapa")
student3 = Student(name="Himal" , age=24 , address = "Gauriganj")
session.add(student1)
session.add(student2)
session.add(student3)

session.commit()

2026-01-18 08:52:37,202 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-18 08:52:37,240 INFO sqlalchemy.engine.Engine INSERT INTO students (name, address, age) VALUES (?, ?, ?) RETURNING id
2026-01-18 08:52:37,256 INFO sqlalchemy.engine.Engine [generated in 0.00018s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('Ram', 'Dharan', 20)
2026-01-18 08:52:37,304 INFO sqlalchemy.engine.Engine INSERT INTO students (name, address, age) VALUES (?, ?, ?) RETURNING id
2026-01-18 08:52:37,305 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('Angel', 'Jhapa', 26)
2026-01-18 08:52:37,318 INFO sqlalchemy.engine.Engine INSERT INTO students (name, address, age) VALUES (?, ?, ?) RETURNING id
2026-01-18 08:52:37,323 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('Himal', 'Gauriganj', 24)
2026-01-18 08:52:37,336 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
# TO extract the data from the database

data = session.query(Student).all()
for row in data:
    # print(row.id , row.name , row.address)
    
    if row.address == "Jhapa":
        print(row.name)

2025-12-31 13:22:36,686 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.address AS students_address, students.age AS students_age 
FROM students
2025-12-31 13:22:36,689 INFO sqlalchemy.engine.Engine [cached since 40.21s ago] ()
Angel
Angel


In [5]:
# to filter out the datas

data = session.query(Student).filter_by(address="Jhapa")
for row in data:
    print(row.id , row.name , row.address)

2026-01-18 08:52:44,721 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-18 08:52:44,733 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.address AS students_address, students.age AS students_age 
FROM students 
WHERE students.address = ?
2026-01-18 08:52:44,734 INFO sqlalchemy.engine.Engine [generated in 0.00163s] ('Jhapa',)
2 Angel Jhapa
